# Transcription de paroles (Speech To Text)

**IMPORTANT :** Créez une ressource *Voix* ou *Speech* et renseignez `REGION` avec votre region. Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [1]:
from getpass import getpass

CLE = getpass("Clé")
REGION = 'francecentral'  # Votre région

## Appel API

In [3]:
import requests

def transcrire(langue, fichier):
  return requests.post(
    url=f'https://{REGION}.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language={langue}&format=detailed',
    headers = {
      'Ocp-Apim-Subscription-Key': CLE,
      'Content-Type': 'audio/wav'
    },
    data=open(fichier, 'rb').read()
  ).json()

## Exemples

In [4]:
transcrire('fr-fr', 'data/FR_1.WAV.wav')

{'RecognitionStatus': 'Success',
 'Offset': 2300000,
 'Duration': 288200000,
 'NBest': [{'Confidence': 0.79342836,
   'Lexical': "les développeurs professionnels usent et abusent joyeusement de ces commentaires et ils ont de très bonnes raisons pour cela premièrement les commentaires rendent le code hyper lisible et hyper compréhensible c'est exactement ce qui se passe ici dans norme live point c s regardez chaque style c s s est ici commenté c'est ça qui rend ce fichier si populaire et si simple à utiliser mais ce n'est pas tout les commentaires rendent également possible le travail en équipe",
   'ITN': "les développeurs professionnels usent et abusent joyeusement de ces commentaires et ils ont de très bonnes raisons pour cela premièrement les commentaires rendent le code hyper lisible et hyper compréhensible c'est exactement ce qui se passe ici dans norme live point CS regardez chaque style CSS est ici commenté c'est ça qui rend ce fichier si populaire et si simple à utiliser mais c

In [5]:
transcrire('pt-br', 'data/BP_1.WAV.wav')

{'RecognitionStatus': 'Success',
 'Offset': 3700000,
 'Duration': 236600000,
 'NBest': [{'Confidence': 0.7611689,
   'Lexical': 'esta tag seu repetir esta tag e seu conteúdo podem ser colocados no cabeçalho ou red do documento html dentro do corpo ou bare do elemento ou até mesmo depois onde você a coloca depende de vários fatores incluindo quando você quer que o script seja executado ou quando',
   'ITN': 'esta tag seu repetir esta tag e seu conteúdo podem ser colocados no cabeçalho ou red do documento html dentro do corpo ou bare do elemento ou até mesmo depois onde você a coloca depende de vários fatores incluindo quando você quer que o script seja executado ou quando',
   'MaskedITN': 'esta tag seu repetir esta tag e seu conteúdo podem ser colocados no cabeçalho ou red do documento html dentro do corpo ou bare do elemento ou até mesmo depois onde você a coloca depende de vários fatores incluindo quando você quer que o script seja executado ou quando',
   'Display': 'Esta tag seu re